In [6]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from tqdm.notebook import tqdm
tqdm.pandas()

import torch
import torchmetrics as tm
import seaborn as sns
from matplotlib import pyplot as plt
import re
import datetime

import warnings
warnings.filterwarnings("ignore")

In [7]:
# ll = ["pierre-roberta-bce_2022-04-01T15-53-15-127273",
# "pierre-roberta-pwbce_2022-04-01T00-16-50-635795",
# "pierre-roberta-fl_2022-04-01T20-43-05-082657",
# "pierre-roberta-pwfl_2022-04-01T20-44-35-459744",
# "pierre-albert-pwbce_2022-04-02T08-52-15-316727",
# "pierre-distilbert-pwbce_2022-04-02T08-51-40-340808",
# "pierre-hatebert-pwbce_2022-04-02T14-16-41-524853",
# "pierre-xlmroberta-pwbce_2022-04-02T14-48-21-276717",
# "pierre-xlnet-pwbce_2022-04-02T18-31-29-083854",
# "pierre-bertweet-pwbce_2022-04-03T03-03-05-377047",
# "pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T03-07-30-490184",
# "pierre-freezed-glove-resnet44-pwbce_2022-04-03T10-58-39-295726",
# "pierre-glove-bigru-pwbce_2022-04-03T18-21-04-474521",
# "pierre-glove-bilstm-pwbce_2022-04-03T20-20-15-399610",
# "glove-cct-pwbce_2022-04-03T22-49-02-960730",
# "pierre-unfreezed-glove-resnet56-pwbce_2022-04-06T13-32-28-490973"]

In [8]:
# for m, _, _, _ in SELECTED_SESSION_INFO:
#     if m not in ll:
#         print(m)

In [9]:
# def find(l):
#     for m, _, _, _ in SELECTED_SESSION_INFO:
#         if m==l:
#             return True
#     return False

In [10]:
# for l in ll:
#     if not find(l):
#         print(l)

In [11]:
SESSION_DIR_PATH = Path("../../session")
DATA_DIR_PATH = Path("../../data")
SESSION_DIR_PATH = Path("../../session")
TRAIN_DATASET_PATH = DATA_DIR_PATH.joinpath("jigsaw2019-train.csv")
TEST_DATASET_PATH = DATA_DIR_PATH.joinpath("jigsaw2019-test.csv")
LABEL_LIST = ['toxicity', 'obscene', 'sexual_explicit',
            'identity_attack', 'insult', 'threat']
IDENTITY_LIST = ['male', 'female', 'transgender', 'other_gender', 'heterosexual',
                'homosexual_gay_or_lesbian', 'bisexual','other_sexual_orientation',
                'christian', 'jewish', 'muslim', 'hindu','buddhist', 'atheist',
                'other_religion', 'black', 'white', 'asian', 'latino',
                'other_race_or_ethnicity', 'physical_disability',
                'intellectual_or_learning_disability',
                'psychiatric_or_mental_illness','other_disability']
SELECTED_IDENTITY_LIST = ['male', 'female', 'black', 'white', 'homosexual_gay_or_lesbian',
                    'christian', 'jewish', 'muslim', 'psychiatric_or_mental_illness']

SELECTED_SESSION_INFO = [('pierre-roberta-bce_2022-04-01T15-53-15-127273', 'BERT', 134_088_961, None),
                        ('pierre-roberta-pwbce_2022-04-01T00-16-50-635795', 'BERT', 134_088_961, None),
                        ('pierre-roberta-fl_2022-04-01T20-43-05-082657', 'BERT', 134_088_961, None),
                        ('pierre-roberta-pwfl_2022-04-01T20-44-35-459744', 'BERT', 134_088_961, None),
                        ('pierre-albert-pwbce_2022-04-02T08-52-15-316727', 'BERT', 21_142_278, None),
                        ('pierre-distilbert-pwbce_2022-04-02T08-51-40-340808', 'BERT', 75_821_574, None),
                        ('pierre-hatebert-pwbce_2022-04-02T14-16-41-524853', 'BERT', 118_940_934, None),
                        ('pierre-xlmroberta-pwbce_2022-04-02T14-48-21-276717', 'BERT', 287_502_342, None),
                        ('pierre-xlnet-pwbce_2022-04-02T18-31-29-083854', 'XLNet', 126_177_030, None),
                        ('pierre-bertweet-pwbce_2022-04-03T03-03-05-377047', 'BERT', 144_358_662, None),
                        ('pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T03-07-30-490184', 'CNN', 26_274_138, 0.071897), # Compute on pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T16-38-13-195724
                        ('pierre-freezed-glove-resnet44-pwbce_2022-04-03T10-58-39-295726', 'CNN', 658_038, None),
                        ('pierre-glove-bigru-pwbce_2022-04-03T18-21-04-474521', 'RNN', 27_665_706, None),
                        ('pierre-glove-bilstm-pwbce_2022-04-03T20-20-15-399610', 'RNN', 28_348_106, None),
                        ('glove-cct-pwbce_2022-04-03T22-49-02-960730', 'RNN', 62_132_971, None),
                        ('pierre-unfreezed-glove-resnet56-pwbce_2022-04-06T13-32-28-490973', 'RNN', 26_468_570, None)
                        ]

Il manquait des lignes de log pour calculer le batch time d'inférence pour `pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T03-07-30-490184`, donc j'ai pris une session avec exactement le même modèle `pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T16-38-13-195724` pour le calculer. 

In [12]:
validation_dataset_namespace = lambda session_name: f"{session_name}.jigsaw2019-validation.csv"
log_namespace = lambda session_name: f"{session_name}.loguru.log"
metric_namespace = lambda session_name: f"{session_name}.metric.json"
test_prediction_namespace = lambda session_name: f"{session_name}.test.csv"
validation_prediction_namespace = lambda session_name: f"{session_name}.validation.csv"

In [13]:
# Get Batch Time
def compte_batch_time(session_info_list):
    for i, (session_name, model_type, params, manual_batch_time) in tqdm(enumerate(session_info_list)):
        if manual_batch_time is not None:
            continue
        log_path = SESSION_DIR_PATH.joinpath(session_name).joinpath(log_namespace(session_name))
        log_line_regex = r"^(?P<date>[ 0-9-:.]*) \| TRACE    \| __main__:evaluation:(\d)+ - batch_id=(?P<batch_id>\d+)$"
        with open(log_path, "r") as f:
            line_list = f.readlines()
        line_list = map(lambda line: re.match(log_line_regex, line), line_list)
        line_list = filter(lambda line: line is not None, line_list)
        line_list = map(lambda m: (int(m.group("batch_id")), datetime.datetime.fromisoformat(m.group("date"))), line_list)
        line_list = list(line_list)

        if len(line_list) == 0:
            SELECTED_SESSION_INFO[i] = (session_name, model_type, params, None)
            print(session_name, ": not complet")
            continue
        first_batch_id, first_batch_date = line_list[0]
        last_batch_id, last_batch_date = line_list[-1]
        batch_number = last_batch_id - first_batch_id
        batch_deltatime = last_batch_date - first_batch_date
        batch_time = (batch_deltatime/batch_number).total_seconds()
        SELECTED_SESSION_INFO[i] = (session_name, model_type, params, batch_time)

        

In [14]:
compte_batch_time(SELECTED_SESSION_INFO)

0it [00:00, ?it/s]

In [15]:
print(SELECTED_SESSION_INFO)

[('pierre-roberta-bce_2022-04-01T15-53-15-127273', 'BERT', 134088961, 0.153711), ('pierre-roberta-pwbce_2022-04-01T00-16-50-635795', 'BERT', 134088961, 0.15284), ('pierre-roberta-fl_2022-04-01T20-43-05-082657', 'BERT', 134088961, 0.154159), ('pierre-roberta-pwfl_2022-04-01T20-44-35-459744', 'BERT', 134088961, 0.154602), ('pierre-albert-pwbce_2022-04-02T08-52-15-316727', 'BERT', 21142278, 0.177142), ('pierre-distilbert-pwbce_2022-04-02T08-51-40-340808', 'BERT', 75821574, 0.089415), ('pierre-hatebert-pwbce_2022-04-02T14-16-41-524853', 'BERT', 118940934, 0.171139), ('pierre-xlmroberta-pwbce_2022-04-02T14-48-21-276717', 'BERT', 287502342, 0.152607), ('pierre-xlnet-pwbce_2022-04-02T18-31-29-083854', 'XLNet', 126177030, 0.242603), ('pierre-bertweet-pwbce_2022-04-03T03-03-05-377047', 'BERT', 144358662, 0.1533), ('pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T03-07-30-490184', 'CNN', 26274138, 0.071897), ('pierre-freezed-glove-resnet44-pwbce_2022-04-03T10-58-39-295726', 'CNN', 658038, 0.073

```python
[('pierre-roberta-bce_2022-04-01T15-53-15-127273', 'BERT', 134088961, 0.153711),
('pierre-roberta-pwbce_2022-04-01T00-16-50-635795', 'BERT', 134088961, 0.15284),
('pierre-roberta-fl_2022-04-01T20-43-05-082657', 'BERT', 134088961, 0.154159),
('pierre-roberta-pwfl_2022-04-01T20-44-35-459744', 'BERT', 134088961, 0.154602),
('pierre-albert-pwbce_2022-04-02T08-52-15-316727', 'BERT', 21142278, 0.177142),
('pierre-distilbert-pwbce_2022-04-02T08-51-40-340808', 'BERT', 75821574, 0.089415),
('pierre-hatebert-pwbce_2022-04-02T14-16-41-524853', 'BERT', 118940934, 0.171139),
('pierre-xlmroberta-pwbce_2022-04-02T14-48-21-276717', 'BERT', 287502342, 0.152607),
('pierre-xlnet-pwbce_2022-04-02T18-31-29-083854', 'XLNet', 126177030, 0.242603),
('pierre-bertweet-pwbce_2022-04-03T03-03-05-377047', 'BERT', 144358662, 0.1533),
('pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T03-07-30-490184', 'CNN', 26274138, 0.071897),
('pierre-freezed-glove-resnet44-pwbce_2022-04-03T10-58-39-295726', 'CNN', 658038, 0.073078),
('pierre-glove-bigru-pwbce_2022-04-03T18-21-04-474521', 'RNN', 27665706, 0.03195),
('pierre-glove-bilstm-pwbce_2022-04-03T20-20-15-399610', 'RNN', 28348106, 0.034753)]
```

In [ ]:
[('pierre-roberta-bce_2022-04-01T15-53-15-127273', 'BERT', 134088961, 0.153711),
('pierre-roberta-pwbce_2022-04-01T00-16-50-635795', 'BERT', 134088961, 0.15284),
('pierre-roberta-fl_2022-04-01T20-43-05-082657', 'BERT', 134088961, 0.154159),
('pierre-roberta-pwfl_2022-04-01T20-44-35-459744', 'BERT', 134088961, 0.154602),
('pierre-albert-pwbce_2022-04-02T08-52-15-316727', 'BERT', 21142278, 0.177142),
('pierre-distilbert-pwbce_2022-04-02T08-51-40-340808', 'BERT', 75821574, 0.089415),
('pierre-hatebert-pwbce_2022-04-02T14-16-41-524853', 'BERT', 118940934, 0.171139),
('pierre-xlmroberta-pwbce_2022-04-02T14-48-21-276717', 'BERT', 287502342, 0.152607),
('pierre-xlnet-pwbce_2022-04-02T18-31-29-083854', 'XLNet', 126177030, 0.242603),
('pierre-bertweet-pwbce_2022-04-03T03-03-05-377047', 'BERT', 144358662, 0.1533),
('pierre-unfreezed-glove-resnet44-pwbce_2022-04-03T03-07-30-490184', 'CNN', 26274138, 0.071897),
('pierre-freezed-glove-resnet44-pwbce_2022-04-03T10-58-39-295726', 'CNN', 658038, 0.073078),
('pierre-glove-bigru-pwbce_2022-04-03T18-21-04-474521', 'RNN', 27665706, 0.03195),
('pierre-glove-bilstm-pwbce_2022-04-03T20-20-15-399610', 'RNN', 28348106, 0.034753),
('glove-cct-pwbce_2022-04-03T22-49-02-960730', 'RNN', 62132971, 0.204585*4),
('pierre-unfreezed-glove-resnet56-pwbce_2022-04-06T13-32-28-490973', 'RNN', 26468570, 0.08401)]